In [1]:
import pandas as pd
import requests
import json
import base64

In [2]:
df = pd.read_csv('diabetes.csv')
df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
27523,Female,28.0,0,0,No Info,40.10,6.0,90,0
27524,Male,31.0,0,0,current,19.81,6.2,155,0
27525,Male,14.0,0,0,never,27.32,6.5,130,0
27526,Male,53.0,0,0,never,29.40,6.6,155,0


In [3]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })


In [6]:
input = df.iloc[50, :8].to_dict()
input

{'gender': 'Male',
 'age': 57.0,
 'hypertension': 0,
 'heart_disease': 0,
 'smoking_history': 'never',
 'bmi': 27.32,
 'HbA1c_level': 6.1,
 'blood_glucose_level': 155}

In [10]:
import tensorflow as tf

json_data = prepare_json(input)

endpoint = "https://ragged-voyage-production.up.railway.app/v1/models/diabetes-classification-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "No diabetes" if prediction_value < 0.5 else "Diabetes"
else:
    result = "Error: No predictions found in response."

print(result)

No diabetes
